In [1]:
# SPACEFACTORY
# Title        : Video Differencing
# Description  : Captures video content from camera or file and performs basic
#                CV2 image processing to highlight and count actors (moving objects)
# Author       : David Malott
# Date         : 10 February 2018
# Version      : v1.0


# IMPORTS
from skimage.measure import compare_ssim
import argparse
import imutils
import cv2
import numpy as np
import time
import datetime

# GLOBAL PARAMETERS

# set font and graphics
font = cv2.FONT_HERSHEY_SIMPLEX  

# set image size scaling factor
scale = 1.0

# set area threshold of actors
areaThresh = 10

# set colors in sequence b, g, r
boxColor = (0, 0, 255)
textColor = (0, 0, 255)

# MAIN CODE

# Initialize video capture object from camera
#img = cv2.VideoCapture(0)

# Initialize video capture object from file
vcap = cv2.VideoCapture('ski-test-2.mp4')

# get image parameters
width = vcap.get(3)  
height = vcap.get(4) 

# apply scale factor, then print image parameters
height = int(height * scale)
width = int(width * scale)

print ("Width: " , width)
print ("Height: " , height)


# define codec and create video writer object
fourcc = cv2.VideoWriter_fourcc(*'MJPG')
cameraOut = cv2.VideoWriter('camera.mov', fourcc, 15.0, (width, height))
displayOut = cv2.VideoWriter('display.mov', fourcc, 15.0, (width, height))

# initialize the first frame in the video stream
prevImg = None

count = 0

# Loop until keypress Esc or end of video
while True:

    # Capture video frame-by-frame
    grabbed, img = vcap.read()

    # if frame could not be grabbed (reached end of video) then end loop
    if not grabbed:
        break
    
    #process only every 5 frames
    #count += 1
    #if count % 5 != 0:
    #    continue
    
    # Resize frame and make a copy of the original before processing
    img = cv2.resize(img, None, fx=scale, fy=scale, interpolation=cv2.INTER_AREA)
    
    # Makes copies to show the original and the final output
    imgCamera = img.copy()
    imgDisplay = img.copy()
    
    # convert frame to grayscale then blur it to reduce sensitivity
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.GaussianBlur(img, (11, 11), 0)
    
    # if the first frame is None, initialize it and continue
    if prevImg is None:
        prevImg = img
        continue    
        
    # compute the absolute difference between the current frame and previous frame,
    # then create threshold image of the differences
    imgDiff = cv2.absdiff(prevImg, img)
    imgThresh = cv2.threshold(imgDiff, 15, 255, cv2.THRESH_BINARY)[1]

    # compute the Structural Similarity Index (SSIM) between the two
    # images, ensuring that the difference image is returned
    #(score, diff) = compare_ssim(grayA, grayB, full=True)
    #diff = (diff * 255).astype("uint8")
    #print("SSIM: {}".format(score))
    
    # dilate thresholded image to fill in holes, then find contours on thresholded image
    imgThresh = cv2.dilate(imgThresh, None, iterations=2)
    contours = cv2.findContours(imgThresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = contours[0] if imutils.is_cv2() else contours[1]
    
    #set variable to count number of actors (aka, moving objects) in the frame
    numActors = len(contours)

    # loop over contours
    for c in contours:
        
        # if contour is too small, continue
        if cv2.contourArea(c) < areaThresh:
            continue
    
        # compute bounding box of contour, then draw it
        (x, y, w, h) = cv2.boundingRect(c)
        cv2.rectangle(imgDisplay, (x, y), (x + w, y + h), boxColor, 1)
    
    # draw text and timestamp on frame
    cv2.putText(imgDisplay, datetime.datetime.now().strftime("%A %d %B %Y %I:%M:%S%p"), 
                (10, 10), font, 0.35, textColor, 1)
    cv2.putText(imgDisplay, "Actors in Scene: {}".format(numActors), (10, 30), 
                font, 0.5, textColor, 1)
        
    # Display frames
    cv2.imshow('Threshold', imgThresh)
    cv2.imshow('Display', imgDisplay)
    
    # assign the original image as previous, and delay
    prevImg = img
    
    # write images to video writer object
    displayOut.write(imgDisplay)
    cameraOut.write(imgCamera)
     
    # break if Esc key has been pressed
    c = cv2.waitKey(1)
    if c == 27:
        break
        
# when done, release the video capture object
vcap.release()

# release the video writer objects
displayOut.release()
cameraOut.release()

# Close all active windows
cv2.destroyAllWindows()      



Width:  960
Height:  720
